In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=26795605ba517bdde61a5a9ae712358fa4d36f180f0d1edaeff92fe241291b21
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [3]:
import pyspark
from pyspark.sql import SparkSession

In [4]:

spark = SparkSession.Builder().appName('merging').getOrCreate()

In [5]:
sales_df = spark.read.load('/content/sales_data_set.csv',format = 'csv',header = True,inferSchema = True)
stores_df = spark.read.load('/content/stores_data_set.csv',format = 'csv',header = True,inferSchema = True)
features_df = spark.read.load('/content/Features_data_set.csv',format = 'csv',header = True,inferSchema = True)

print('Number of rows : ',sales_df.count()), display(sales_df)
print('Number of rows : ',stores_df.count()), display(stores_df)
print('Number of rows : ',features_df.count()), display(features_df)

Number of rows :  421570


DataFrame[Store: int, Dept: int, Date: string, Weekly_Sales: double, IsHoliday: boolean]

Number of rows :  45


DataFrame[Store: int, Type: string, Size: int]

Number of rows :  8190


DataFrame[Store: int, Date: string, Temperature: double, Fuel_Price: double, MarkDown1: string, MarkDown2: string, MarkDown3: string, MarkDown4: string, MarkDown5: string, CPI: string, Unemployment: string, IsHoliday: boolean]

(None, None)

In [7]:
merge_df = sales_df.join(stores_df, on = 'Store', how = 'inner')
print('Number of rows : ',merge_df.count()), display(merge_df)

merge_df1 = merge_df.join(features_df, on = ['Store','Date','IsHoliday'], how = 'inner')
print('Number of rows : ',merge_df1.count()), display(merge_df1)

Number of rows :  421570


DataFrame[Store: int, Dept: int, Date: string, Weekly_Sales: double, IsHoliday: boolean, Type: string, Size: int]

Number of rows :  421570


DataFrame[Store: int, Date: string, IsHoliday: boolean, Dept: int, Weekly_Sales: double, Type: string, Size: int, Temperature: double, Fuel_Price: double, MarkDown1: string, MarkDown2: string, MarkDown3: string, MarkDown4: string, MarkDown5: string, CPI: string, Unemployment: string]

(None, None)

In [8]:
merge_df1.createOrReplaceTempView('merge_df')

In [9]:
query_df = spark.sql('''select year(Date) as Year, Store, `Type` as Store_Type ,Dept, Temperature, Fuel_Price, CPI, Unemployment,
                        sum(Weekly_Sales) as Total_Sales from merge_df
                        group by Year, Store, Store_Type, Dept, Temperature, Fuel_Price, CPI, Unemployment''')

print('Number of rows : ',query_df.count()), display(query_df)

Number of rows :  421570


DataFrame[Year: int, Store: int, Store_Type: string, Dept: int, Temperature: double, Fuel_Price: double, CPI: string, Unemployment: string, Total_Sales: double]

(None, None)

In [10]:
# Write CSV file with column header (column names)
query_df.write.option("header",True).csv("/FileStore/tables/consolidate/merged_df")